# 0. Imports

In [0]:
# Imports
import pandas as pd

In [0]:
# Load data set
df = pd.read_csv("https://drive.switch.ch/index.php/s/jBrNwNiAGh7LcXX/download")

The EDA showed that the changes to the legislation had an impact on inspections. Those changes included new violations, terminology modification and more, which would lead to perturbation in the results of this project. Consequently, only the inspections that occured before July 1st, 2018 will be considered for this project.

In [0]:
# Date conversion
df["Inspection Date"] =  pd.to_datetime(df["Inspection Date"])

In [0]:
df = df[df["Inspection Date"] < "2018-07-01"]

# 1. Augmentation using the original dataframe

In [0]:
# Extract week day, month and year from inspection date
df["Year"] = pd.DatetimeIndex(df['Inspection Date']).year
df["Month"] = pd.DatetimeIndex(df['Inspection Date']).month
df["Weekday"] = pd.DatetimeIndex(df['Inspection Date']).dayofweek

# 2. Augmentation using the NOAA Weather history

In [0]:
# Load data
df_weather = pd.read_csv("https://drive.switch.ch/index.php/s/ui6Zr1v2vzPlieH/download")

In [0]:
# Column selection
df_weather_zip = df_weather[["DATE", "TMAX"]]

In [318]:
# Compute the mean maximum temparture over the last 3 days for each day.
df_weather_zip["MeanMaxTemp3Days"] = (1.0/3.0)*(df_weather_zip.TMAX.shift(1) + df_weather_zip.TMAX.shift(2) + df_weather_zip.TMAX.shift(3))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [319]:
# Example : 1/3 * ((-2.1) + (-4.9) + (2.8)) = -1.4
# which corresponds to 1/3 * (TMAX(2009-12-15) + TMAX(2009-12-16) + TMAX(2009-12-17)) = MeanMaxTemp3Days(2009-12-18)
df_weather_zip.head(10)

,DATE,TMAX,MeanMaxTemp3Days
0,2009-12-15,-2.1,NaN
1,2009-12-16,-4.9,NaN
2,2009-12-17,2.8,NaN
3,2009-12-18,3.3,-1.400000
4,2009-12-19,1.1,0.400000
5,2009-12-20,-0.5,2.400000
6,2009-12-21,-1.0,1.300000
7,2009-12-22,0.0,-0.133333
8,2009-12-23,0.0,-0.500000
9,2009-12-24,2.2,-0.333333


In [0]:
# Date conversion
df_weather_zip["DATE"] =  pd.to_datetime(df_weather_zip["DATE"])

In [0]:
df = df.merge(df_weather_zip, left_on = "Inspection Date", right_on = "DATE")

In [0]:
df.to_csv("augmented_food_inspections.csv")